<a href="https://colab.research.google.com/github/odaren/tensorflow-sample/blob/main/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
# MNISTデータセットをロードして準備します。サンプルを整数から浮動小数点数に変換します。
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
# 層を積み重ねてtf.keras.Sequentialモデルを構築します。訓練のためにオプティマイザと損失関数を選びます。
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [4]:
# モデルはそれぞれの標本について、クラスごとに"ロジット"や"対数オッズ比"と呼ばれるスコアを算出します。
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.14574765, -0.55691135, -0.6422523 ,  0.8041989 ,  0.3464385 ,
         0.5816304 ,  1.0275643 , -0.03857642, -0.7461361 , -0.09347074]],
      dtype=float32)

In [5]:
# tf.nn.softmax 関数はクラスごとにこれらのロジットを "確率" に変換します。
tf.nn.softmax(predictions).numpy()

array([[0.09013502, 0.04464087, 0.04098921, 0.17412265, 0.11016724,
        0.13937825, 0.2177016 , 0.07496225, 0.0369448 , 0.07095815]],
      dtype=float32)

In [6]:
# losses.SparseCategoricalCrossentropy 損失関数は、ロジットと True の インデックスに関するベクトルを入力にとり、それぞれの標本についてクラスごとに損失のスカラーを返します。
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [7]:
# この訓練されていないモデルはランダムに近い確率 (それぞれのクラスについて 1/10) を出力します、最初の損失は -tf.log(1/10) ~= 2.3 に近い値になるはずです。
loss_fn(y_train[:1], predictions).numpy()

1.9705639

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [9]:
# Model.fit メソッドは損失を最小化するようにモデルのパラメータを調整します。
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2972 - accuracy: 0.9125
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1466 - accuracy: 0.9570
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1084 - accuracy: 0.9672
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0861 - accuracy: 0.9738
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0757 - accuracy: 0.9763


In [10]:
# Model.evaluate メソッドはモデルの性能を検査します。これには通常 "検証用データセット" または "テストデータセット" を用います。
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0721 - accuracy: 0.9770 - 517ms/epoch - 2ms/step


[0.07205630838871002, 0.9769999980926514]

In [11]:
# モデルが確率を返すようにしたい場合には、モデルをラップしてソフトマックス関数を適用してください
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[8.1447901e-08, 8.0834850e-09, 4.1684784e-06, 1.2924064e-04,
        6.0003646e-10, 2.3858037e-08, 3.4895906e-13, 9.9985790e-01,
        8.8197589e-07, 7.7101413e-06],
       [2.1427460e-09, 2.2427742e-03, 9.9775201e-01, 4.4592257e-06,
        6.3546271e-13, 4.4286580e-07, 2.5862528e-09, 1.5716279e-12,
        4.1195511e-07, 1.7575834e-12],
       [1.1748323e-06, 9.9924052e-01, 1.8019653e-04, 1.1448031e-05,
        4.4386466e-05, 2.3336493e-06, 1.5947158e-05, 2.7615472e-04,
        2.2179312e-04, 6.1162459e-06],
       [9.9994445e-01, 2.1998985e-09, 3.1806892e-05, 3.2927542e-08,
        1.7852381e-07, 9.7813633e-08, 1.5887113e-05, 2.3406242e-06,
        1.0423523e-09, 5.2067621e-06],
       [7.7585741e-05, 5.8791564e-08, 5.0445724e-05, 9.7723705e-09,
        9.9279881e-01, 2.3214669e-07, 2.0047943e-05, 1.8577024e-04,
        6.4134196e-07, 6.8662995e-03]], dtype=float32)>